Credit goes to this youtuber Pradip Nichite:
1.   https://blog.futuresmart.ai/langchain-sql-agents-openai-llms-query-database-using-natural-language
2.   https://youtu.be/VG9KYCS0-8E?feature=shared


### Install libraries

In [ ]:
# pip install -U langchain-community langchain openai pymysql -q

### Connect to OpenAI

In [2]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

### Connect to the SQL Database

In [3]:
from langchain.sql_database import SQLDatabase
db_user= userdata.get('db_user')
db_password = userdata.get('db_password')
db_host = userdata.get('db_host')
db_name = userdata.get('db_name')
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

### Set up the LLM, toolkit, and sql agent :

In [5]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
import json

# Initialize the language model
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# Initialize the SQLDatabaseToolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

prompt_template = """
You are a helpful assistant that responds to SQL queries with structured output.

Given the input: "{input}"

Please return the result in the following format:
{{
  "Thought process": "<insert thought process>",
  "SQL query": "<insert SQL query>",
  "Final Answer": "<insert final answer>"
}}

{agent_scratchpad}
"""

# Create the prompt template with placeholders for both `query` and `agent_scratchpad`
formatted_prompt = PromptTemplate(
    input_variables=["query", "agent_scratchpad"],
    template=prompt_template
)

# Set up the agent with the custom prompt template
agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type="openai-tools",
    verbose=True,
    prompt=formatted_prompt
)


user_question = "Find the top 5 products with the highest total sales revenue"

# Run the agent
response = agent_executor.invoke(user_question)


# Format output as JSON
parsed_output = json.loads(response["output"])
formatted_output = json.dumps(parsed_output, indent=4)
print(formatted_output)

<ipython-input-5-a3476ac5cd4f>:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo")




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`
responded: I will first need to check the database schema to understand the tables and columns involved in this query. Let's start by listing the tables in the database.

customers, employees, offices, orderdetails, orders, payments, productlines, products
Invoking: `sql_db_schema` with `{'table_names': 'products, orderdetails'}`



CREATE TABLE orderdetails (
	`orderNumber` INTEGER NOT NULL, 
	`productCode` VARCHAR(15) NOT NULL, 
	`quantityOrdered` INTEGER NOT NULL, 
	`priceEach` DECIMAL(10, 2) NOT NULL, 
	`orderLineNumber` SMALLINT NOT NULL, 
	PRIMARY KEY (`orderNumber`, `productCode`), 
	CONSTRAINT orderdetails_ibfk_1 FOREIGN KEY(`orderNumber`) REFERENCES orders (`orderNumber`), 
	CONSTRAINT orderdetails_ibfk_2 FOREIGN KEY(`productCode`) REFERENCES products (`productCode`)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from orderdetails table:
orderNumber	productCode	

### Error Handling

In [6]:
user_question = "Describe the PurchaseDetails table"
# Run the agent
response = agent_executor.invoke(user_question)
# Format output as JSON
parsed_output = json.loads(response["output"])
formatted_output = json.dumps(parsed_output, indent=4)
print(formatted_output)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_schema` with `{'table_names': 'PurchaseDetails'}`


Error: table_names {'PurchaseDetails'} not found in database
Invoking: `sql_db_list_tables` with `{}`


customers, employees, offices, orderdetails, orders, payments, productlines, products
Invoking: `sql_db_list_tables` with `{}`


customers, employees, offices, orderdetails, orders, payments, productlines, products{
  "Thought process": "The PurchaseDetails table was not found in the database, so I will first check the list of available tables to ensure the table name is correct.",
  "SQL query": "sql_db_list_tables",
  "Final Answer": "The available tables in the database are: customers, employees, offices, orderdetails, orders, payments, productlines, products"
}

> Finished chain.
{
    "Thought process": "The PurchaseDetails table was not found in the database, so I will first check the list of available tables to ensure the table name is correct.",
    "SQL query":